In [6]:
import cv2
cv2.__version__

'4.5.2'

In [12]:
import logging
import numpy as np

In [13]:

# Homomorphic filter class
class HomomorphicFilter:


    def __init__(self, a = 0.5, b = 1.5):
        self.a = float(a)
        self.b = float(b)

    # Filters
    def __butterworth_filter(self, I_shape, filter_params):
        P = I_shape[0]/2
        Q = I_shape[1]/2
        U, V = np.meshgrid(range(I_shape[0]), range(I_shape[1]), sparse=False, indexing='ij')
        Duv = (((U-P)**2+(V-Q)**2)).astype(float)
        H = 1/(1+(Duv/filter_params[0]**2)**filter_params[1])
        return (1 - H)

    def __gaussian_filter(self, I_shape, filter_params):
        P = I_shape[0]/2
        Q = I_shape[1]/2
        H = np.zeros(I_shape)
        U, V = np.meshgrid(range(I_shape[0]), range(I_shape[1]), sparse=False, indexing='ij')
        Duv = (((U-P)**2+(V-Q)**2)).astype(float)
        H = np.exp((-Duv/(2*(filter_params[0])**2)))
        return (1 - H)

    # Methods
    def __apply_filter(self, I, H):
        H = np.fft.fftshift(H)
        I_filtered = (self.a + self.b*H)*I
        return I_filtered

    def filter(self, I, filter_params, filter='butterworth', H = None):

        #  Validating image
        if len(I.shape) != 2:
            raise Exception('Improper image')

        # Take the image to log domain and then to frequency domain 
        I_log = np.log1p(np.array(I, dtype="float"))
        I_fft = np.fft.fft2(I_log)

        # Filters
        if filter=='butterworth':
            H = self.__butterworth_filter(I_shape = I_fft.shape, filter_params = filter_params)
        elif filter=='gaussian':
            H = self.__gaussian_filter(I_shape = I_fft.shape, filter_params = filter_params)
        elif filter=='external':
            print('external')
            if len(H.shape) != 2:
                raise Exception('Invalid external filter')
        else:
            raise Exception('Selected filter not implemented')
        
        # Apply filter on frequency domain then take the image back to spatial domain
        I_fft_filt = self.__apply_filter(I = I_fft, H = H)
        I_filt = np.fft.ifft2(I_fft_filt)
        I = np.exp(np.real(I_filt))-1
        return np.uint8(I)
# End of class HomomorphicFilter

if __name__ == "__main__":
    import cv2
    # Code parameters
    path_in = 'C:/Users/pasindu/OneDrive/Pictures/'
    path_out = 'C:/Users/pasindu/OneDrive/Music/'
    img_path = 'dent.jpg'

    # Derived code parameters
    img_path_in = path_in + img_path
    img_path_out = path_out + 'dent.png'

    # Main code
    img = cv2.imread(img_path_in)[:, :, 0]
    homo_filter = HomomorphicFilter(a = 0.75, b = 1.25)
    img_filtered = homo_filter.filter(I=img, filter_params=[30,2])
    cv2.imwrite(img_path_out, img_filtered)

In [17]:
#Closing operation of morphological operation
#closing = cv.morphologyEx(img, cv.MORPH_CLOSE, kernel)
kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(img_path_out, cv2.MORPH_CLOSE, kernel)
cv2.imshow('Grayscale',closing)
cv2.waitKey()
cv2.destroyAllWindows()

error: OpenCV(4.5.2) :-1: error: (-5:Bad argument) in function 'morphologyEx'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'
